In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import os
import time
import tkinter.font as tkFont

def stitch_with_sift(map_1, map_2):
    if map_1 is None or map_2 is None:
        print("Error: One of the input images is None.")
        return None
    
    sift = cv2.SIFT_create()
    
    # Detect keypoints and descriptors
    kp1, des1 = sift.detectAndCompute(map_1, None)
    kp2, des2 = sift.detectAndCompute(map_2, None)

    if des1 is None or des2 is None:
        print("Error: No keypoints detected in one of the images.")
        return map_1

    # FLANN-based matcher
    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=10), dict(checks=50))
    matches = flann.knnMatch(des1, des2, k=2)

    # Lowe's ratio test
    good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]
    print(f"Found {len(good_matches)} good matches")

    if len(good_matches) > 40:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])

        # Compute Homography
        M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

        if M is not None:
            h1, w1 = map_1.shape[:2]
            h2, w2 = map_2.shape[:2]

            # Transform corners to find bounding box
            corners_2 = np.float32([[0, 0], [w2, 0], [w2, h2], [0, h2]]).reshape(-1, 1, 2)
            transformed_corners = cv2.perspectiveTransform(corners_2, M)

            # Get bounding box
            min_x = int(min(transformed_corners[:, 0, 0].min(), 0))
            max_x = int(max(transformed_corners[:, 0, 0].max(), w1))
            min_y = int(min(transformed_corners[:, 0, 1].min(), 0))
            max_y = int(max(transformed_corners[:, 0, 1].max(), h1))

            # Create translation matrix
            translation = np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]])

            # Warp second image onto new canvas
            new_width = max_x - min_x
            new_height = max_y - min_y
            warped_map_2 = cv2.warpPerspective(map_2, translation @ M, (new_width, new_height))

            # Place first image onto the canvas
            warped_map_1 = np.zeros_like(warped_map_2)
            warped_map_1[-min_y:h1 - min_y, -min_x:w1 - min_x] = map_1

            # Create mask for blending
            mask_1 = (warped_map_1 > 0).astype(np.uint8)
            mask_2 = (warped_map_2 > 0).astype(np.uint8)
            overlap = mask_1 * mask_2

            # Alpha blending
            alpha = 0.5
            blended = np.where(overlap > 0,
                            cv2.addWeighted(warped_map_1, alpha, warped_map_2, 1 - alpha, 0),
                            warped_map_1 + warped_map_2)

            return blended.astype(np.uint8)
        else:
            print("Failed to compute homography.")
            return map_1
    else:
        print("Not enough good matches to stitch images.")
        return map_1

def select_images():
    file_paths = filedialog.askopenfilenames(title="Select Images", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    return sorted(list(file_paths))  # Sort filenames alphanumerically

def stitch_images():
    file_paths = select_images()
    if not file_paths:
        messagebox.showerror("Error", "No images selected.")
        return

    maps = [cv2.imread(file) for file in file_paths]
    if all(m is not None for m in maps):
        merged_image = maps[0]
        for i in range(1, len(maps)):
            print(f"Stitching image {i + 1}...")
            merged_image = stitch_with_sift(merged_image, maps[i])
            if merged_image is None:
                messagebox.showerror("Error", f"Stitching failed at image {i + 1}")
                return

        if merged_image is not None:
            output_dir = "stitched_output"
            os.makedirs(output_dir, exist_ok=True)
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            output_path = os.path.join(output_dir, f"stitched_{timestamp}.png")
            cv2.imwrite(output_path, merged_image)
            messagebox.showinfo("Success", f"Merged image saved as {output_path}")

            # Display the merged image
            cv2.imshow("Merged Image", merged_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
    else:
        messagebox.showerror("Error", "Not all map images were loaded correctly.")

def main():
    root = tk.Tk()
    root.title("Image Stitching Tool")

    # Increase font size
    font_style = tkFont.Font(family="Arial", size=14, weight="bold")

    btn_stitch = tk.Button(root, text="Select and Stitch Images", command=stitch_images, font=font_style)
    btn_stitch.pack(pady=20)

    root.mainloop()

if __name__ == "__main__":
    main()

Stitching image 2...
Found 4007 good matches
Stitching image 2...
Found 3660 good matches
